In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
from keras.layers import Dense
from keras import Input
#from keras.engine import Model
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_dir = 'C:/Users/USER/Desktop/spcup_2022/audiofile/'

In [9]:
X = []
Y = []
categories = ["0","1","2","3","4"]
num_classes = len(categories)

for filename in os.listdir(f'C:/Users/USER/Desktop/spcup_2022/audiofile/'):
    audioname = f'C:/Users/USER/Desktop/spcup_2022/audiofile/{filename}'
    y, sr = librosa.load(audioname, duration=1, sr = 16000)  # 모두 1초까지만 자름
    stft = librosa.stft(y, n_fft=512, hop_length=384)  # 일반적으로 1/4 가량 겹치게 함
    # phase, mag 추출
    magnitude, phase = librosa.magphase(stft)
    # mag를 넘파이 배열로
    magnitude = np.array(magnitude)
    # radian값으로 변환
    phase = np.angle(phase)
    phase = np.array(phase)
    # normalization_magnitude
    scaler = StandardScaler()
    scaler.fit(magnitude)
    magnitude = scaler.transform(magnitude)
    # normalization_phase
    scaler = StandardScaler()
    scaler.fit(phase)
    phase = scaler.transform(phase)
    ## 두 개 결합
    magphase = np.concatenate((magnitude,phase)) 
    X.append(magphase)

for idex, category in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    for j in range(1000):
        Y.append(label)

X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(5000, 514, 42)
(5000, 5)


In [10]:
X[0].shape, X[1].shape, X[318].shape

((514, 42), (514, 42), (514, 42))

In [11]:
X_train = np.concatenate((X[0:800],X[1000:1800],X[2000:2800],X[3000:3800],X[4000:4800]), axis = 0)
X_test = np.concatenate((X[800:1000],X[1800:2000],X[2800:3000],X[3800:4000],X[4800:5000]), axis = 0)
Y_train = np.concatenate((Y[0:800],Y[1000:1800],Y[2000:2800],Y[3000:3800],Y[4000:4800]), axis = 0)
Y_test = np.concatenate((Y[800:1000],Y[1800:2000],Y[2800:3000],Y[3800:4000],Y[4800:5000]), axis = 0)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4000, 514, 42)
(1000, 514, 42)
(4000, 5)
(1000, 5)


In [32]:
# CNN 모델에 들어가려면 4차원으로 바꿔줘야함
X_train = X_train.reshape(-1,257,84,1)
X_test = X_test.reshape(-1,257,84,1)

In [33]:
from keras.models import Sequential
from keras.layers import Flatten, Conv2D, MaxPooling2D

In [34]:
model = Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(257, 84, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dense(32, activation='relu'))
model.add(Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))


In [35]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
classifier = model.fit(X_train,
                    Y_train,
                    epochs=10,
                    batch_size=64)

Epoch 1/10
63/63 [==============================] - 59s 929ms/step - loss: 1.0473 - accuracy: 0.6442
Epoch 2/10
63/63 [==============================] - 58s 926ms/step - loss: 0.1794 - accuracy: 0.9352
Epoch 3/10
63/63 [==============================] - 58s 922ms/step - loss: 0.0472 - accuracy: 0.9887
Epoch 4/10
63/63 [==============================] - 58s 926ms/step - loss: 0.0106 - accuracy: 0.9990
Epoch 5/10
63/63 [==============================] - 58s 927ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 6/10
63/63 [==============================] - 58s 924ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 7/10
63/63 [==============================] - 58s 923ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 8/10
63/63 [==============================] - 59s 932ms/step - loss: 5.9782e-04 - accuracy: 1.0000
Epoch 9/10
63/63 [==============================] - 58s 926ms/step - loss: 4.3655e-04 - accuracy: 1.0000
Epoch 10/10
63/63 [==============================] - 59s 931ms/step - loss: 3.4441e

In [ ]:
plt.plot(classifier.history['loss'])
#plt.plot(classifier.history['accuracy'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show()

In [37]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)
print('테스트 정확도:', test_acc)

32/32 - 4s - loss: 0.1920 - accuracy: 0.9280 - 4s/epoch - 138ms/step
테스트 정확도: 0.9279999732971191


In [38]:
prediction = model.predict(X_test)

In [39]:
predicted_classes = np.argmax(prediction, axis = 1)
print(predicted_classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1
 1 1 1 1 2 1 1 1 2 1 1 2 1 2 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 2 1 2 2 1 1 1 1 1 1 1 1 2 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 1 1 2 2 2 1 2 2 2 2 2 2
 2 2 2 1 2 2 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 1 2 2 2 2 2 2 1 

In [20]:
labels = np.argmax(Y_test, axis=1)
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [40]:
labels.shape, predicted_classes.shape

((1000,), (1000,))

In [41]:
print(np.where(predicted_classes[0] != labels[0]))

(array([], dtype=int64),)


In [42]:
index = np.where(predicted_classes != labels)
print(index)

(array([ 74, 208, 212, 244, 252, 263, 267, 270, 272, 275, 284, 290, 303,
       305, 307, 308, 317, 319, 323, 339, 340, 345, 346, 369, 380, 382,
       401, 403, 404, 405, 418, 428, 432, 433, 437, 447, 450, 458, 459,
       466, 469, 476, 485, 492, 531, 543, 555, 561, 565, 571, 573, 577,
       579, 581, 583, 584, 585, 598, 616, 632, 659, 662, 686, 721, 722,
       732, 733, 740, 744, 745, 779, 793], dtype=int64),)


In [43]:
len(index[0])

72

In [44]:
index[0][0]

74

In [45]:
X_filename = []
for filename in os.listdir(f'C:/Users/USER/Desktop/spcup_2022/audiofile/'):
    audioname = f'C:/Users/USER/Desktop/spcup_2022/audiofile/{filename}'
    X_filename.append(audioname)
X_filename = np.array(X_filename)

In [46]:
len(X_filename)

5000

In [47]:
X_test_filename = np.concatenate((X_filename[800:1000],X_filename[1800:2000],X_filename[2800:3000],X_filename[3800:4000],X_filename[4800:5000]), axis = 0)

In [48]:
len(X_test_filename)

1000

In [49]:
X_test_filename[0]

'C:/Users/USER/Desktop/spcup_2022/audiofile/0_819.wav'

In [31]:
# 잘못 예측한 오디오 파일들 
wrong_predictions = []

for i in range(len(index[0])):
    wrong_predictions.append(X_test_filename[index[0][i]])
    print(X_test_filename[index[0][i]])
    
print(len(wrong_predictions))

C:/Users/USER/Desktop/spcup_2022/audiofile/1_820.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_823.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_828.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_83.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_838.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_847.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_859.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_863.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_864.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_866.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_867.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_875.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_876.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_88.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_884.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_894.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_897.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_90.wav
C:/Users/USER/Desktop/spcup_2022/audiofile/1_900.

In [ ]:
# 잘못 예측한거 plot
for audio in wrong_predictions:
    y, sr = librosa.load(audio, duration=1)  # 모두 1초까지만 자름
    stft = np.abs(librosa.stft(y, n_fft=512, hop_length=384))   # 일반적으로 1/4 가량 겹치게 함
    spectogram = np.abs(stft)
    plt.figure(figsize=(10,5))
    plt.title(audio)
    plt.plot(spectogram) 

In [ ]:
# 잘못 예측한 오디오 파일들 모은 폴더 생성
import shutil
import os
 
file_source = 'C:/Users/USER/Desktop/spcup_2022/audiofile/'
file_destination = 'C:/Users/USER/Desktop/wrong_prediction'
 
get_files = os.listdir(file_source)
 

for i in range(len(wrong_predictions)):
    print(wrong_predictions[i][43:])
    for g in get_files:
        if g == wrong_predictions[i][43:]:
            shutil.move(file_source + g, file_destination)